#  RAG
###  RAG

The problem with RAG is basically text chunking.

Therefore a pattern we can implement is synthetic readers. Synthetic readers know what page will answer the question of a user.

Therefore all we need is a mapping of question to relevant content.

We then vectorize the questions and we have a high dimensional set of potential questions a reader would ask about the textbook.

Once a reader asks a question they are mapped to the relevant information.

This is contrasting from taking the prompt -> distilling to intents -> performing similarity search on a vector library

In [ ]:
! pip3 install azure-search-documents==11.4.0b11
! pip3 install openai
! pip3 install python-dotenv
! pip3 install PyMuPDF
! pip3 install tiktoken
! pip3 install ratelimit
! pip3 install tenacity

In [1]:
# Import required libraries  
import os  
import json  
import openai
import fitz
import tiktoken
import re
import hashlib
import time
from ratelimit import limits, sleep_and_retry
import dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  

from azure.search.documents.indexes.models import (  
    ExhaustiveKnnVectorSearchAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    VectorSearchVectorizer,
    VectorSearchVectorizerKind,
    AzureOpenAIParameters,
    AzureOpenAIVectorizer,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
    ExhaustiveKnnVectorSearchAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings, 
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    AzureOpenAIParameters,
    AzureOpenAIVectorizer,
)  
  
# Configure environment variables  
dotenv.load_dotenv(dotenv_path="../.env", override=True)
service_endpoint = str(os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT"))
print(service_endpoint)
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION") 
model: str = "text-embedding-ada-002" 
credential = AzureKeyCredential(key)

https://plutolearning-acs-eastus2-basic.search.windows.net


In [2]:
def chunk_text(text: str, limit: int, encoding: tiktoken.Encoding):
    """Split a string into parts of given size without breaking words.
    
    Args:
        text (str): Text to split.
        limit (int): Maximum number of tokens per part.
        encoding (tiktoken.Encoding): Encoding to use for tokenization.
        
    Returns:
        list[str]: List of text parts.
        
    """
    tokens = encoding.encode(text)
    parts = []
    text_parts = []
    current_part = []
    current_count = 0

    for token in tokens:
        current_part.append(token)
        current_count += 1

        if current_count >= limit:
            parts.append(current_part)
            current_part = []
            current_count = 0

    if current_part:
        parts.append(current_part)

    # Convert the tokenized parts back to text
    for part in parts:
        text = [
            encoding.decode_single_token_bytes(token).decode("utf-8", errors="replace")
            for token in part
        ]
        text_parts.append("".join(text))

    return text_parts


In [ ]:
def read_pdf(file):
    text = []
    with fitz.open(file) as doc:
        for page in doc:
            text.append(page.get_text())
    return ''.join(text)